In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
sns.set_theme()

In [ ]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
from src.utils import get_config, read_file

In [ ]:
config = get_config.read_yaml()
symbols = config['data']['symbols']
symbols

In [ ]:
symbol = symbols[7]
symbol

In [ ]:
data = read_file.read_raw_data(symbol)
data

In [ ]:
symbol = symbols[0]
symbol

In [ ]:
data = read_file.read_raw_data(symbol)
data

In [ ]:
# define a function to plot various datafields
def plot_df(df, kind=None):
    if kind is None:
        df.plot(figsize = (15,8), fontsize=12) # plots the graph with specified plot and font size
    else:
        df.plot(kind=kind, figsize = (15,8), fontsize=12, bins=100)
    plt.legend(fontsize = 12) # shows legends
    plt.show() # makes the plot visible

In [ ]:
plot_df(data['close'])

In [ ]:
plot_df(data['volume'])

In [ ]:
data =  read_file.read_preprocessed_training_data("SOL/USDT")
data